In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
import os
import random 

from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
# read in original data
train_data = "hida_workspace/trainSet/trainSet.txt"
test_data = "hida_workspace/testSet/testSet.txt"

df_train = pd.read_csv(train_data)
df_train_length_idx = len(df_train)
df_test = pd.read_csv(test_data)

# Merge the two datasets
train_test = [df_train, df_test]
df_train_test = pd.concat(train_test)
# df_train_test['Prognosis'].loc[df_train_test['Prognosis'] == '<undefined>'] = np.nan
df_train_test.loc[df_train_test['Prognosis'] == '<undefined>', 'Prognosis'] = np.nan

# all variables in the dataset (incl. outcome)
variables = list(df_train_test.columns[3:])


# read in csv from Tobias (imputed via catboost
df_imputed_train = pd.read_csv("Results/Tobias/imputed_train.csv")
df_imputed_test = pd.read_csv("Results/Tobias/imputed_test.csv")


df_for_LGBM = df_train.copy()
df_for_LGBM["Prognosis"] = df_for_LGBM["Prognosis"].astype("category").cat.codes
df_for_LGBM = df_for_LGBM[variables]


In [4]:
# get predictions from LightGBM not imputed:
LGBM_features = variables.copy()
LGBM_features.remove("Prognosis")

train_valid_split = 0.9
sep_index = int(863 * train_valid_split)

X_train = df_for_LGBM[LGBM_features].iloc[0:sep_index]
y_train = df_for_LGBM["Prognosis"].iloc[0:sep_index]


X_test = df_for_LGBM[LGBM_features].iloc[sep_index:]
y_test = df_for_LGBM["Prognosis"].iloc[sep_index:]
image_names = df_train["ImageFile"].iloc[sep_index:]

# fit model on training data
model = LGBMClassifier()
model.fit(X_train.values, y_train.values)


# make predictions for test data
y_pred = model.predict(X_test.values)
LGBM_results = dict(predictions_LGBM=list(y_pred), images=image_names.to_list(), targets=y_test.values)
LGBM_results_df = pd.DataFrame(LGBM_results)

In [6]:
# get predictions from LGBM imputed:
LGBM_features = variables.copy()
LGBM_features.remove("Prognosis")


train_valid_split = 0.9
sep_index = int(863 * train_valid_split)

X_train = df_imputed_train[LGBM_features].iloc[0:sep_index]
y_train = df_imputed_train["Prognosis"].iloc[0:sep_index]

X_test = df_imputed_train[LGBM_features].iloc[sep_index:]
y_test = df_imputed_train["Prognosis"].iloc[sep_index:]
image_names = df_train["ImageFile"].iloc[sep_index:]





# fit model on training data
model = LGBMClassifier()
model.fit(X_train.values, y_train.values)


# make predictions for test data
y_pred = model.predict(X_test.values)
LGBM_imputed_results = dict(predictions_LGBM_imputed=list(y_pred), images=image_names.to_list())
LGBM_imputed_results_df = pd.DataFrame(LGBM_imputed_results)

In [7]:
LGBM_imputed_results_df

,predictions_LGBM_imputed,images
0,1.0,P_678.png
1,1.0,P_673.png
2,0.0,P_669.png
3,1.0,P_675.png
4,1.0,P_640.png
...,...,...
82,1.0,P_1_12.png
83,1.0,P_1_8.png
84,0.0,P_1_10.png
85,1.0,P_1_26.png


In [8]:
LGBM_results_df

,predictions_LGBM,images,targets
0,1,P_678.png,1
1,0,P_673.png,1
2,0,P_669.png,0
3,1,P_675.png,1
4,1,P_640.png,1
...,...,...,...
82,1,P_1_12.png,1
83,1,P_1_8.png,1
84,0,P_1_10.png,0
85,1,P_1_26.png,1


In [9]:
# stack predicions:
LGBM_imputed_results_df = LGBM_imputed_results_df.set_index("images")
LGBM_results_df = LGBM_results_df.set_index("images")

all_predictions = pd.concat([LGBM_imputed_results_df, LGBM_results_df], axis=1)

In [10]:
all_predictions

,predictions_LGBM_imputed,predictions_LGBM,targets
images,,,
P_678.png,1.0,1,1
P_673.png,1.0,0,1
P_669.png,0.0,0,0
P_675.png,1.0,1,1
P_640.png,1.0,1,1
...,...,...,...
P_1_12.png,1.0,1,1
P_1_8.png,1.0,1,1
P_1_10.png,0.0,0,0


In [11]:
(all_predictions['predictions_LGBM_imputed']==all_predictions['predictions_LGBM']).sum()

74

In [12]:
(all_predictions['predictions_LGBM_imputed']== all_predictions['targets']).sum()

63

In [13]:
(all_predictions['predictions_LGBM']== all_predictions['targets']).sum()

66